# CS771 Assignment 2 Submission for Adarsh Pal (180032)

In [1]:
import numpy as np
import numpy.linalg as lin

In [2]:
def gradient_descent(gradient,init_,learn_rate, n_iter=50, tol=1e-06):
    x = init_
    for _ in range(n_iter):
        delta = -learn_rate*gradient(x)
        if np.all(np.abs(delta) <= tol):
            break
        x += delta
    return round(x*1000)/1000

# Question 1(a)
Clearly, for $f(x)=x^2+3x+4$, we get $f'(x)=2x+3$ and $f''(x)=2 > 0$. Hence, we see that $f(x)$ is convex and so, we will be converging to its global minima, no matter what initial values we take, as long as the learn_rate is small enough that we do not overshoot.<br>
For $f(x)=x^4-3x^2+2x$, we get $f'(x)=4x^3-6x+2$ and $f''(x)=12x^2-6$. Clearly, this is not a convex function as $f''(x)$ is not positive for every $x$ and so, optimal minima will depend on the initial values chosen.

In [3]:
# The local minima is also the global minima since the function is convex
print("Local Minima for x^2+3x+4=", gradient_descent(lambda x:2*x+3,init_=-10,learn_rate=0.01,n_iter=10000))  
# The local minima is not the global minima as function is not convex
print("Local Minima for x^4-3x^2+2x with init=-5: ", gradient_descent(lambda x:4*(x**3)-6*x+2,init_=-5,learn_rate=0.001,n_iter=10000))
print("Local Minima for x^4-3x^2+2x with init=5: ", gradient_descent(lambda x:4*(x**3)-6*x+2,init_=5,learn_rate=0.001,n_iter=10000))

Local Minima for x^2+3x+4= -1.5
Local Minima for x^4-3x^2+2x with init=-5:  -1.366
Local Minima for x^4-3x^2+2x with init=5:  1.0


# Question 1(b)
Let us define <br>
$w$ =
\begin{bmatrix}
b \\
a
\end{bmatrix}<br>
$X$ =
\begin{bmatrix}
1& x_{1} \\
1& x_{2} \\
\vdots \\
1& x_{n}
\end{bmatrix}<br>
$y$ =
\begin{bmatrix}
 y_{1} \\
 y_{2} \\
\vdots \\
y_{n}
\end{bmatrix}, where n= number of training examples<br>
Then we need to minimize $f(w)=\frac{1}{n} \sum_{i=1}^n (y_i-X_i w)^2$, which can be written as<br>
$f(w)=\frac{1}{n} ||y-Xw||_2^2=\frac{1}{n} (y-Xw)^T (y-Xw)=\frac{1}{n} (y^T-w^TX^T)(y-Xw)\\ \Rightarrow f(w)=\frac{1}{n}(y^Ty-y^TXw-w^TX^Ty+w^TX^TXw)$<br>
Hence, $\nabla f(w)=\frac{1}{n}(-(y^TX)^T-X^Ty+2X^TXw)=\frac{1}{n}(-2X^Ty+2X^TXw)$<br>
Hence, one step of gradient descent becomes:<br>
$w^{(t+1)}=w^{(t)}-\frac{\alpha}{n} \nabla f(w^{(t)})$ where $\alpha=$ learning rate.<br>
$\Rightarrow w^{(t+1)}=w^{(t)}-\frac{\alpha}{n} (-2X^Ty+2X^TXw)$

In [4]:
# This is the gradient function for linear regression
#w is the column vector [b,a]
def linear_regression_gradient(y,X,w):
    n=y.shape[0]
    y=np.reshape(y,(n,1))
    #Adding a column of 1s to X
    X=np.hstack((np.ones((n,1)),X.reshape(n,1)))
    #Finding the gradient according to the formula above
    gradient=(1/n)*(-2*np.matmul(np.transpose(X),y)+2*np.matmul(np.matmul(np.transpose(X),X),w))
    return gradient

In [5]:
#w is the column vector [b,a]
def linear_regression(gradient,y,X,init_a,init_b,learn_rate, n_iter=50, tol=1e-06):
    #Initialize w to [init_b,init_a]
    w=np.array([init_b,init_a])
    w=np.reshape(w,(2,1))
    for _ in range(n_iter):
        delta=-learn_rate*gradient(y,X,w)
        if np.all(np.abs(delta) <= tol):
            break
        w += delta
    w=np.squeeze(w)
    return (w[1],w[0])

# Question 1 (c)

In [6]:
#Generation of data
np.random.seed(0)
X=2.5*np.random.randn(10000)+1.5
res=1.5*np.random.randn(10000)
y=2+0.3*X+res

In [7]:
# Finding optimal values of a and b using gradient descent
(a,b)=linear_regression(linear_regression_gradient,y,X,init_a=1.,init_b=1.,learn_rate=0.1,n_iter=10000,tol=1e-06)
print("a=",a,"b=",b)

a= 0.2953197350589738 b= 2.0232822054084147


# Question 1 (d)
In Minibatch Gradient Descent, we will just be randomly taking k training examples $(x_i$ $_j$,$y_i$ $_j$), $j=1,2,...,k$ and apply gradient descent on them. Hence,<br>
$w$ =
\begin{bmatrix}
b \\
a
\end{bmatrix}<br>
$X_{batch}$ = \begin{bmatrix}
1& x_{i_1} \\
1& x_{i_2} \\
\vdots \\
1& x_{i_k}
\end{bmatrix}<br>
$y_{batch}$ = \begin{bmatrix}
y_{i_1} \\
y_{i_2} \\
\vdots \\
y_{i_k}
\end{bmatrix}<br>
and the gradient descent step becomes $w^{(t+1)}=w^{(t)}-\frac{\alpha}{k} (-2X_{batch}^Ty_{batch}+2X_{batch}^TX_{batch}w)$

In [14]:
#Minibatch Gradient Descent Implementation
# Here k refers to the batch size
def minibatch_gradient_descent(gradient,y,X,k,init_a,init_b,learn_rate, n_iter=50, tol=1e-06):
    w=np.array([init_b,init_a])
    w=np.reshape(w,(2,1))
    for _ in range(n_iter):
        # Choosing k random training examples
        choose= np.random.choice(X.shape[0], k, replace=False)
        Xsgd=X[choose]
        ysgd=y[choose]
        delta=-learn_rate*gradient(ysgd,Xsgd,w)
        if np.all(np.abs(delta) <= tol):
            break
        w+=delta
    w=np.squeeze(w)
    return (w[1],w[0])

In [15]:
# Taking k=1000
(a,b)=minibatch_gradient_descent(linear_regression_gradient,y,X,k=1000,init_a=1.,init_b=1.,learn_rate=0.1,n_iter=10000,tol=1e-06)
print("a=",a,"b=",b)

a= 0.28581950363309866 b= 2.067062816995244


# Question 1 (e) Comparing the performance of Minibatch Gradient Descent vs Normal Gradient Descent

In [10]:
#Choosing batches of size 1,2,4,8,16,32,64,128,256,512,1024,2048,4096,8192
batches= 2**np.arange(0,14)
for i in batches:
  print(i,end=" ")
  # Let's look at the times taken for learn_rate=0.1 and tol=1e-06
  %timeit minibatch_gradient_descent(linear_regression_gradient,y,X,k=i,init_a=1.,init_b=1.,learn_rate=0.1,n_iter=10000,tol=1e-06)

1 1 loop, best of 5: 2.65 s per loop
2 1 loop, best of 5: 2.66 s per loop
4 1 loop, best of 5: 2.66 s per loop
8 1 loop, best of 5: 2.68 s per loop
16 1 loop, best of 5: 2.65 s per loop
32 1 loop, best of 5: 2.69 s per loop
64 1 loop, best of 5: 2.67 s per loop
128 1 loop, best of 5: 2.72 s per loop
256 1 loop, best of 5: 2.71 s per loop
512 1 loop, best of 5: 2.76 s per loop
1024 1 loop, best of 5: 2.81 s per loop
2048 1 loop, best of 5: 2.93 s per loop
4096 1 loop, best of 5: 3.21 s per loop
8192 1 loop, best of 5: 5.82 s per loop


In [11]:
# Let's look at the time taken by gradient descent for learn_rate=0.1 and tol=1e-06
%timeit linear_regression(linear_regression_gradient,y,X,init_a=1.,init_b=1.,learn_rate=0.1,n_iter=10000,tol=1e-6)

100 loops, best of 5: 12.2 ms per loop


Thus, we can see that for learn_rate=0.1 and tol=1e-06, Linear Regression runs faster than Minibatch Gradient Descent for all batch sizes. This is mainly because Minibatch Gradient Descent although takes lesser time for a single iteration of the for loop as compared to Gradient Descent, it takes a longer time to converge owing to lower tolerance and higher step size. We are not moving in the direction of steepest descent (since we are taking a subset of the total training examples only). As a result, minibatch gradient descent takes more .

In [16]:
#Choosing batches of size 1,2,4,8,16,32,64,128,256,512,1024,2048,4096,8192
batches= 2**np.arange(0,14)
for i in batches:
  print(i,end=" ")
  # Let's look at the times taken for learn_rate=1e-3 and tol=1e-04
  %timeit minibatch_gradient_descent(linear_regression_gradient,y,X,k=i,init_a=1.,init_b=1.,learn_rate=1e-3,n_iter=10000,tol=1e-04)

1 10 loops, best of 5: 12.2 ms per loop
2 The slowest run took 16.49 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 16.8 ms per loop
4 The slowest run took 25.02 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 11.3 ms per loop
8 1 loop, best of 5: 126 ms per loop
16 10 loops, best of 5: 130 ms per loop
32 1 loop, best of 5: 165 ms per loop
64 1 loop, best of 5: 138 ms per loop
128 1 loop, best of 5: 230 ms per loop
256 1 loop, best of 5: 268 ms per loop
512 1 loop, best of 5: 283 ms per loop
1024 1 loop, best of 5: 357 ms per loop
2048 1 loop, best of 5: 421 ms per loop
4096 1 loop, best of 5: 503 ms per loop
8192 1 loop, best of 5: 1.02 s per loop


In [17]:
%timeit linear_regression(linear_regression_gradient,y,X,init_a=1.,init_b=1.,learn_rate=1e-3,n_iter=10000,tol=1e-04)

1 loop, best of 5: 300 ms per loop


Thus, we see that as we decrease the step size, the minibatch gradient descent gives a better performance than normal gradient descent for smaller values. This may be attributed to the smaller step size and the higher tolerance which prevents minibatch gradient descent from overshooting and oscillating around the minima. Also, we observe that the time taken is increasing with increasing batch size.

If we look at the error terms $|a_{predicted}-0.3|+|b_{predicted}-2.0|$ for the various batch sizes, they are as follows:

In [18]:
def batch_size_losses():
  batches=2**np.arange(0,14)
  ab_loss=list()
  for i in batches:
    (a,b)=minibatch_gradient_descent(linear_regression_gradient,y,X,k=i,init_a=1.,init_b=1.,learn_rate=0.1,n_iter=10000,tol=1e-06)
    ab_loss.append(np.abs(a-0.3)+np.abs(b-2))
    print("batch_size=",i,"absolute_loss=",ab_loss[-1],"a=",a,"b=",b)

In [19]:
batch_size_losses()

batch_size= 1 absolute_loss= 11.042689343976118 a= 0.06477362657146202 b= -8.80746297054758
batch_size= 2 absolute_loss= 7.72096669697454 a= 3.3152971971996887 b= -2.705669499774851
batch_size= 4 absolute_loss= 0.6053647561562137 a= -0.05313525281895548 b= 1.7477704966627419
batch_size= 8 absolute_loss= 0.19812140478571422 a= 0.1587206105738348 b= 1.943157984640451
batch_size= 16 absolute_loss= 1.2240908515725932 a= -0.6592777340278722 b= 1.735186882455279
batch_size= 32 absolute_loss= 0.1880761826774575 a= 0.16962743233541652 b= 2.057703615012874
batch_size= 64 absolute_loss= 0.28584315404288363 a= 0.40767951150703013 b= 2.1781636425358535
batch_size= 128 absolute_loss= 0.20254988190233889 a= 0.11577891286204464 b= 2.0183287947643835
batch_size= 256 absolute_loss= 0.0752590002319305 a= 0.37121228079374213 b= 2.0040467194381884
batch_size= 512 absolute_loss= 0.03785932329357994 a= 0.2785325585466836 b= 2.0163918818402635
batch_size= 1024 absolute_loss= 0.07275445456638738 a= 0.24664074

Clearly, if we look at the losses for different batch sizes, we find out that any batch size >= 128 is fine, but as the batch size increases, more and more time is taken. Also, note that for batch size=1, the larger step size coupled with higher gradient is causing it to move towards infinity. Let us look at the validation error for different batch sizes as well:

In [20]:
# Splitting into training and validation data
msk = np.random.rand(X.shape[0]) < 0.8
X_train = X[msk]
X_validation = X[~msk]
y_train = y[msk]
y_validation = y[~msk]

In [21]:
def batch_size_RMSE(X_train,X_validation,y_train,y_validation):
  batches=2**np.arange(0,13)
  sq_loss=list()
  for i in batches:
    (a,b)=minibatch_gradient_descent(linear_regression_gradient,y_train,X_train,k=i,init_a=1.,init_b=1.,learn_rate=0.1,n_iter=10000,tol=1e-06)
    rmse=1/X_validation.shape[0]*lin.norm(y_validation-a*X_validation-b)
    print("batch_size=",i,"RMSE=",rmse)

In [22]:
batch_size_RMSE(X_train,X_validation,y_train,y_validation)

batch_size= 1 RMSE= 7.495120736093498e+50
batch_size= 2 RMSE= 0.056767840102433305
batch_size= 4 RMSE= 0.03556595947386352
batch_size= 8 RMSE= 0.03803357839997114
batch_size= 16 RMSE= 0.03419313971680664
batch_size= 32 RMSE= 0.03461965916434229
batch_size= 64 RMSE= 0.03361741225358691
batch_size= 128 RMSE= 0.03505726287031953
batch_size= 256 RMSE= 0.03346582284103738
batch_size= 512 RMSE= 0.0334902608076817
batch_size= 1024 RMSE= 0.03348280478592348
batch_size= 2048 RMSE= 0.03343964548475806
batch_size= 4096 RMSE= 0.03344639906604042


Even using a validation set and finding the RMSE does not differentiate between batch sizes by much. Thus, we might end up getting minimal improvements w.r.t. accuracy, but the process will take more time with increasing batch size. Hence, taking time taken, squared_loss and RMSE into account, we see that a batch_size of 64 to 128 could be optimal.

In [170]:
(a,b)=minibatch_gradient_descent(linear_regression_gradient,y,X,k=128,init_a=1.,init_b=1.,learn_rate=0.1,n_iter=10000,tol=1e-06)
print("a=",a,"b=",b)

a= 0.18952200202830877 b= 2.044251411849159


# Question 2: Bayesian Analysis
Let us define a few terms: <br>
$P(cold)$:= Probability that someone has a cold <br>
$P(fever)$:= Probability that someone has a fever <br>
$P(cough)$:= Probability that someone has a cough <br>
$P(lung)$:= Probability that someone has a lung disease <br>
$P(smokes)$:= Probability that someone smokes <br>
(a) The question wants us to find $P(cold \cap fever)$ <br> Clearly, $P(cold \cap fever)=P(fever|cold)P(cold)=0.307*0.02=0.00614$ <br>
(b) The question wants us to find $P(cold | cough)$. <br>
Note that<br>
$P(lung)=P(lung|smokes)*P(smokes)+P(lung|not\ smokes)*P(not\ smokes)$<br>
$=0.1009*0.2+0.001*0.8=0.02098$, and <br>
$P(lung|cold)=P(lung)=0.02098$ (As lung and cold are independent events) and <br>
$P(not\ lung|cold)=P(not\ lung)=1-P(lung)=1-0.02098=0.97902$ (Same reason as above).<br>
Now, $P(cough | cold)$<br>$=P(cough | cold \cap lung)P(lung|cold)+P(cough | cold \cap not\ lung)P(not\ lung|cold)$<br>$=P(cough | cold \cap lung)P(lung)+P(cough | cold \cap not\ lung)P(not\ lung)$<br>$=0.7525*0.02098+0.505*0.97902$<br>$=0.51019255$<br>
Similarly,<br>
$P(cough | not\ cold)$<br>$=P(cough | not\ cold \cap lung)P(lung|not\ cold)+P(cough | not\ cold \cap not\ lung)P(not\ lung|not\ cold)$<br>$=P(cough | not\ cold \cap lung)P(lung)+P(cough | not\ cold \cap not\ lung)P(not\ lung)$<br>$=0.505*0.02098+0.01*0.97902=0.0203851$<br>
Hence, by Bayes' Method,<br>
$P(cold|cough)$<br>$=\frac{P(cough|cold)P(cold)}{P(cough|cold)P(cold)+P(cough|not\ cold)P(not\ cold)}$<br>
=$\frac{0.51019255*0.02}{0.51019255*0.02+0.0203851*0.98}=\frac{0.010203851}{0.010203851+0.019977398}=\frac{0.010203851}{0.030181249}=0.338085776$


# Question 3: MLE for a Multinomial Distribution
Let $X=(X_1,X_2,...,X_k)\sim Mult(n,p_1,p_2,...,p_k)$ such that $X_i$ appears $x_i$ times. $\sum_{j=1}^k x_j=n$, and $\sum_{j=1}^k p_j =1$ Also, $x_i\ge0$ and $p_i\ge0$ for $j \in \{1,2,...,k\}$.
Then,<br>
$L(X_1,X_2,...,X_k | (p_1,p_2,...,p_k))$<br>$
=P(x_1,x_2,...,x_k)| (p_1,p_2,...,p_k))\\
=\frac{n!}{x_1!x_2! ... x_k!} p_1^{x_1} p_2^{x_2} ... p_k^{x_k}$, where $x_k=n-\sum_{j=1}^{k-1} x_j$ and $p_k=1-\sum_{j=1}^{k-1} p_j$<br>$
\Rightarrow log L(X_1,X_2,...,X_k| (p_1,p_2,...,p_k))$<br>$=\log({ \frac{n!}{x_1!x_2! ... x_k!} p_1^{x_1} p_2^{x_2} ... p_k^{x_k}})$<br>$
=\log{\frac{n!}{x_1!x_2! ... x_k!}}+\log{(p_1^{x_1})}+\log{(p_2^{x_2})}+...+\log{(p_k^{x_k})}$<br>$
=\log{\frac{n!}{x_1!x_2! ... x_k!}} +x_1\log{(p_1)}+x_2\log{(p_2)}+...+x_k\log{(p_k)}$<br>
$\therefore NLL(p_1,p_2,...,p_k):= -log L(X_1,X_2,...,X_k| (p_1,p_2,...,p_k))$<br>$=-\log{\frac{n!}{x_1!x_2! ... x_k!}} -x_1\log{(p_1)}-x_2\log{(p_2)}-...-x_k\log{(p_k)}$<br>
Hence, we need to find the values of $(p_1,p_2,...,p_k)$ minimizing $NLL(p_1,p_2,...,p_k)$
This function does not have independent parameters as $\sum_{j=1}^k p_j =1$<br>
Hence, this is a constrained optimization problem with $\sum_{j=1}^k p_j =1$.
Hence, we will have to use a Lagrangian to find the optimal parameters for minimizing $NLL(p_1,p_2,...,p_k)$.<br>
Take $F(p_1,p_2,...,p_k,\lambda):=NLL(p_1,p_2,...,p_k)+\lambda*(\sum_{j=1}^k p_j-1)$<br>
Hence, the optimization problem becomes the primal problem<br>
$\min_{(p_1,p_2,..,p_k)} (\max_{\lambda} (F(p_1,p_2,...,p_k,\lambda)))$<br>
Interchanging the min and the max (which we can do, since $p_is$ and $\lambda$ have independent domains, we will get the dual problem
$\max_{\lambda} (\min_{(p_1,p_2,..,p_k)} (F(p_1,p_2,...,p_k,\lambda)))$<br>
For solving the above problem, we do the following:<br>
For $j \in \{1,2,...,k\}, \frac{\partial F}{\partial p_j}=0 \Rightarrow -\frac{x_j}{p_j}+\lambda=0 \Rightarrow p_j=\frac{x_j}{\lambda}$<br>
Also,<br>
$\sum_{j=1}^k p_j-1=0$<br>$\Rightarrow \lambda(\sum_{j=1}^k p_j-1)=0 $<br>$\Rightarrow (\sum_{j=1}^k \lambda p_j-\lambda)=0$<br>$\Rightarrow (\sum_{j=1}^k x_j-\lambda)=0$<br>$\Rightarrow \lambda=\sum_{j=1}^k x_j=n$<br>$
 \therefore (p_j)_{MLE}=\frac{x_j}{\lambda}=\frac{x_j}{n}$<br>
To show that $NLL(p_1,p_2,...,p_k)$ is semi-convex, we see that<br>
for $i,j \in \{1,2,...,k\}, \frac{\partial (NLL)}{\partial p_j}= -\frac{x_j}{p_j}+\lambda \Rightarrow \frac{\partial^2 (NLL)}{\partial p_j^2}= \frac{x_j}{p_j^2}\ge 0 $ and<br>
$\frac{\partial^2 (NLL)}{\partial p_i \partial p_j}=0$ for $i\neq j$.<br>
Hence the Hessian matrix becomes a diagonal matrix $D$ with diagonal entries $D_{jj}=\frac{x_j}{p_j^2}\ge 0$ Hence, the Hessian is a positive semi-definite matrix since all the eigenvalues of $D$ are non-negative and thus, $NLL(p_1,p_2,...,p_k)$ is semi-convex.